# 1. Inicializando projeto

In [1]:
from dotenv import dotenv_values

config = dotenv_values("../../.env")

pinecone_api_ley = config["PINECONE_API_KEY"]
pinecone_env = config["PINECONE_ENVIRONMENT"]
index_pinecone = config["INDEX_NAME"]

openai_api_key = config["OPENAI_API_KEY"]

## 1.1 Inicializar cliente Pinecone (configurar conexão)

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_ley)
index = pc.Index(index_pinecone)

# Verificando se está tudo ok
print(pc.list_indexes())

c:\Users\irani\Desktop\Personal code\LLM\Using LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{
    "name": "livreto-base-evidencia",
    "metric": "cosine",
    "host": "livreto-base-evidencia-y4phmo4.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "region": "us-east-1",
            "cloud": "aws",
            "read_capacity": {
                "mode": "OnDemand",
                "status": {
                    "state": "Ready",
                    "current_shards": null,
                    "current_replicas": null
                }
            }
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": {
        "embedding_model": "text-embedding-3-small"
    }
}]


# 2. Extrair texto do PDF

In [7]:
from pathlib import Path

path = Path("LangChain/RAG/Data/Livreto_BaseEvidencia_2019.11.21.pdf")
print("Existe?", path.exists())
print("Caminho absoluto:", path.resolve())

Existe? False
Caminho absoluto: C:\Users\irani\Desktop\Personal code\LLM\Using LLM\LangChain\RAG\LangChain\RAG\Data\Livreto_BaseEvidencia_2019.11.21.pdf


In [10]:
import pdfplumber
import json

path = "Data/Livreto_BaseEvidencia_2019.11.21.pdf"
pages=[]
with pdfplumber.open(path) as pdf:
    for i , p in enumerate(pdf.pages, start=1):
        text = p.extract_text() or ""
        pages.append({"page": i, "text": text})

with open("data/pdf_pages_livreto.json","w",encoding="utf-8") as f:
    json.dump(pages,f,ensure_ascii=False,indent=2)

# Limpeza e normalização de texto

In [11]:
import re

def limpar_texto(texto: str) -> str:
    if not texto:
        return ""

    # Remove caracteres estranhos comuns de OCR
    texto = re.sub(r'[^\wÀ-ÿ\s.,;:!?()\-"“”]', ' ', texto)

    # Remove letras soltas em linhas (ex: O\nI\nR\n)
    texto = re.sub(r'(?:\b[A-ZÀ-Ý]\b\s*){3,}', ' ', texto)

    # Normaliza quebras de linha
    texto = re.sub(r'\n{2,}', '\n', texto)
    texto = re.sub(r'\n', ' ', texto)

    # Normaliza espaços
    texto = re.sub(r'\s{2,}', ' ', texto)

    return texto.strip()

def limpar_json_paginas(paginas: list) -> list:
    json_limpo = []

    for pagina in paginas:
        texto_limpo = limpar_texto(pagina.get("text", ""))

        # ignora páginas vazias após limpeza
        if texto_limpo:
            json_limpo.append({
                "page": pagina["page"],
                "text": texto_limpo
            })

    return json_limpo


In [12]:
import json

with open("data/pdf_pages_livreto.json", encoding="utf-8") as f:
    paginas = json.load(f)

json_limpo = limpar_json_paginas(paginas)

with open("data/pdf_pages_livreto_clean.json", "w", encoding="utf-8") as f:
    json.dump(json_limpo, f, ensure_ascii=False, indent=2)


# Dividir texto em pedaços menores (Chunks)

# Embeddings + Ingestão no Pinecone

# Consultas e recuperação dos dados

droga
